

---
# AUTOR : Michael De La Cruz La Rosa
### 20180176@lamolina.edu.pe
### Michael.dlc.lr@gmail.com
---

In [1]:
%reset -f
# omitir warnings :
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Conectando google drive con colab:
#-------------------------------------
from google.colab import drive
drive.mount('/content/drive/')
#-------------------------------------
import os
path = '/content/drive/MyDrive/ColabNotebooks/' # directorio de trabajo
os.chdir(path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Librerías necesarias

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import json
import time
import threading
from pathlib import Path
import stat, getpass

# Instalación de la librería desde github

In [4]:
!pip -q install --upgrade --no-cache-dir git+https://github.com/MaicolDLC/download-merra2.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Creación del archivo .netrc

In [5]:
# Tu carpeta en Drive
drive_netrc = Path(path + ".netrc")

user = input("Usuario Earthdata: ").strip()
pwd = getpass.getpass("Password Earthdata: ")

drive_netrc.write_text(
    f"machine urs.earthdata.nasa.gov login {user} password {pwd}\n"
    f"machine goldsmr4.gesdisc.eosdis.nasa.gov login {user} password {pwd}\n",
    encoding="utf-8",
)

# Permisos en drive (si es que aplica)
try:
    drive_netrc.chmod(stat.S_IRUSR | stat.S_IWUSR)
except Exception:
    pass

root_netrc = Path("/root/.netrc")
root_netrc.write_text(drive_netrc.read_text(encoding="utf-8"), encoding="utf-8")
root_netrc.chmod(stat.S_IRUSR | stat.S_IWUSR)

print("Guardado en:", drive_netrc)
print("Activo en:", root_netrc)

Usuario Earthdata: maicoldlc
Password Earthdata: ··········
Guardado en: /content/drive/MyDrive/ColabNotebooks/.netrc
Activo en: /root/.netrc


# Ejemplo para archivos horarios (MERRA2)

---

### Nota: El catálogo no brinda archivos horarios como tal (24 archivos por día), lo que brinda es 1 archivo con las 24 horas.

---



In [6]:
# Importamos la librería
from merra2_downloader import Merra2Config, Merra2Client

cfg = Merra2Config(
    north=5, south=-20, west=-90, east=-70, # Ingresar zona de interés
    inicio="2023-10-01", # Ingresar fecha en formato YYYY-MM-DD
    fin="2023-11-30",
    producto="M2T1NXAER.5.12.4",  # Ejemplo de producto horario
    variables=[],                 # Vacío: descarga todas las variables
    # variables=["BCEXTTAU", "BCSCATAU"] # Especificando variables
    directorio= path + "merra2_horario", # Directorio de guardado
    max_workers=5, # Hilos de descarga
)

client = Merra2Client()
result = client.download_range(cfg)
print(result)
print(f"Se guardó en: {cfg.directorio}")

[1/3] Error en MERRA2_400.tavg1_2d_aer_Nx.20231130.nc4: 503 Server Error: Service Unavailable for url: https://goldsmr4.gesdisc.eosdis.nasa.gov/thredds/ncss/grid/M2T1NXAER.5.12.4/2023/11/MERRA2_400.tavg1_2d_aer_Nx.20231130.nc4?var=BCANGSTR&var=BCCMASS&var=BCEXTTAU&var=BCFLUXU&var=BCFLUXV&var=BCSCATAU&var=BCSMASS&var=DMSCMASS&var=DMSSMASS&var=DUANGSTR&var=DUCMASS&var=DUCMASS25&var=DUEXTT25&var=DUEXTTAU&var=DUFLUXU&var=DUFLUXV&var=DUSCAT25&var=DUSCATAU&var=DUSMASS&var=DUSMASS25&var=OCANGSTR&var=OCCMASS&var=OCEXTTAU&var=OCFLUXU&var=OCFLUXV&var=OCSCATAU&var=OCSMASS&var=SO2CMASS&var=SO2SMASS&var=SO4CMASS&var=SO4SMASS&var=SSANGSTR&var=SSCMASS&var=SSCMASS25&var=SSEXTT25&var=SSEXTTAU&var=SSFLUXU&var=SSFLUXV&var=SSSCAT25&var=SSSCATAU&var=SSSMASS&var=SSSMASS25&var=SUANGSTR&var=SUEXTTAU&var=SUFLUXU&var=SUFLUXV&var=SUSCATAU&var=TOTANGSTR&var=TOTEXTTAU&var=TOTSCATAU&north=5&west=-90&east=-70&south=-20&horizStride=1&accept=netcdf4-classic&time_start=2023-11-30T00:30:00Z&time_end=2023-11-30T23:30:00Z

DownloadResult(exitosos=61, fallidos=0)
Se guardó en: /content/drive/MyDrive/ColabNotebooks/merra2_horario


# Ejemplo para archivos diurnos (MERRA2_DIURNAL)

In [8]:
# Importamos la librería
from merra2_downloader import Merra2Config, Merra2Client

cfg = Merra2Config(
    north=5, south=-20, west=-90, east=-70, # Ingresar zona de interés
    inicio="2023-10-01", # Ingresar fecha en formato YYYY-MM-DD (mantener -DD)
    fin="2023-11-30",
    producto="M2TUNXAER.5.12.4",  # <---- Cambiar
    variables=[],                 # Vacío: descarga todas las variables
    directorio= path + "merra2_diurno", # Directorio de guardado
    max_workers=5, # Hilos de descarga
)

client = Merra2Client()
result = client.download_range(cfg)
print(result)
print(f"Se guardó en: {cfg.directorio}")

DownloadResult(exitosos=2, fallidos=0)
Se guardó en: /content/drive/MyDrive/ColabNotebooks/merra2_diurno


# Ejemplo para archivos Mensuales (MERRA2_MONTHLY)

In [9]:
# Importamos la librería
from merra2_downloader import Merra2Config, Merra2Client

cfg = Merra2Config(
    north=5, south=-20, west=-90, east=-70, # Ingresar zona de interés
    inicio="2023-10-01", # Ingresar fecha en formato YYYY-MM-DD (mantener -DD)
    fin="2023-11-30",
    producto="M2TMNXAER.5.12.4",  # <---- Cambiar
    variables=[],                 # Vacío: descarga todas las variables
    directorio= path + "merra2_mensual", # Directorio de guardado
    max_workers=5, # Hilos de descarga
)

client = Merra2Client()
result = client.download_range(cfg)
print(result)
print(f"Se guardó en: {cfg.directorio}")

DownloadResult(exitosos=2, fallidos=0)
Se guardó en: /content/drive/MyDrive/ColabNotebooks/merra2_mensual
